In [4]:
import json
import requests
import pandas as pd

In [5]:
url = "https://api.openf1.org/v1/sessions?session_type=Race&year=2023"
response = requests.get(url)
print(f"Status Code: {response.status_code}")

Status Code: 200


In [6]:
data = response.json()
print(json.dumps(data[:3], indent=4))

[
    {
        "meeting_key": 1141,
        "session_key": 7953,
        "location": "Sakhir",
        "date_start": "2023-03-05T15:00:00+00:00",
        "date_end": "2023-03-05T17:00:00+00:00",
        "session_type": "Race",
        "session_name": "Race",
        "country_key": 36,
        "country_code": "BRN",
        "country_name": "Bahrain",
        "circuit_key": 63,
        "circuit_short_name": "Sakhir",
        "gmt_offset": "03:00:00",
        "year": 2023
    },
    {
        "meeting_key": 1142,
        "session_key": 7779,
        "location": "Jeddah",
        "date_start": "2023-03-19T17:00:00+00:00",
        "date_end": "2023-03-19T19:00:00+00:00",
        "session_type": "Race",
        "session_name": "Race",
        "country_key": 153,
        "country_code": "KSA",
        "country_name": "Saudi Arabia",
        "circuit_key": 149,
        "circuit_short_name": "Jeddah",
        "gmt_offset": "03:00:00",
        "year": 2023
    },
    {
        "meeting_key": 11

In [7]:
df = pd.DataFrame(data)
df.head(5)

,meeting_key,session_key,location,date_start,date_end,session_type,session_name,country_key,country_code,country_name,circuit_key,circuit_short_name,gmt_offset,year
0,1141,7953,Sakhir,2023-03-05T15:00:00+00:00,2023-03-05T17:00:00+00:00,Race,Race,36,BRN,Bahrain,63,Sakhir,03:00:00,2023
1,1142,7779,Jeddah,2023-03-19T17:00:00+00:00,2023-03-19T19:00:00+00:00,Race,Race,153,KSA,Saudi Arabia,149,Jeddah,03:00:00,2023
2,1143,7787,Melbourne,2023-04-02T05:00:00+00:00,2023-04-02T07:00:00+00:00,Race,Race,5,AUS,Australia,10,Melbourne,10:00:00,2023
3,1207,9069,Baku,2023-04-29T13:30:00+00:00,2023-04-29T14:00:00+00:00,Race,Sprint,30,AZE,Azerbaijan,144,Baku,04:00:00,2023
4,1207,9070,Baku,2023-04-30T11:00:00+00:00,2023-04-30T13:00:00+00:00,Race,Race,30,AZE,Azerbaijan,144,Baku,04:00:00,2023


In [8]:
# Fetching Code for Multiple Session Keys

session_keys = [7775, 7779, 9476, 9480, 9689, 9693, 9994, 9998, 10002, 10006]

endpoints = {
    "drivers": "https://api.openf1.org/v1/drivers",
    "position": "https://api.openf1.org/v1/position",
    "laps": "https://api.openf1.org/v1/laps",
    "sessions": "https://api.openf1.org/v1/sessions",
}

dataframes = {key: [] for key in endpoints.keys()}

for session_key in session_keys:
    for key, url in  endpoints.items():
        response = requests.get(url, params = {"session_key": session_key }, stream = True)
        if response.status_code == 200:
            df = pd.DataFrame(response.json())
            if not df.empty:
                dataframes[key].append(df)
        else:
            print(f"Failed to fetch {key} data, Status Code: ", response.status_code)

for key in dataframes:
    if dataframes[key]: dataframes[key] = pd.concat(dataframes[key], ignore_index = True)
    else: dataframes[key] = pd.DataFrame()

for key, df in dataframes.items():
    print(f"{key}: {df.shape}")
    

drivers: (200, 12)
position: (7661, 5)
laps: (6444, 16)
sessions: (10, 14)


In [9]:
# Extract Columns of Interest and Optimize Data

driver_subset = dataframes["drivers"][["session_key", "driver_number", "full_name", "team_name", "headshot_url"]]
position_subset = dataframes["position"][["session_key", "driver_number", "position"]]
laps_subset = dataframes["laps"][["session_key", "driver_number", "lap_number"]]
sessions_subset = dataframes["sessions"][["session_key", "circuit_key", "session_type", "location", "date_start"]]


# Optimize dataframes so we only get final positions of each session
position_subset = position_subset.sort_values(["session_key", "driver_number"]).drop_duplicates(subset = ["session_key", "driver_number"], keep = "last")

max_laps = laps_subset.groupby(["session_key", "driver_number"])["lap_number"].max().reset_index()
laps_subset = laps_subset.merge(max_laps, on=["session_key", "driver_number", "lap_number"], how="inner")


# Final array of dataframes before merging
dataframes_subset = [driver_subset, position_subset, laps_subset, sessions_subset]

print(f"Driver subset shape: {driver_subset.shape}")
print(f"Position subset shape: {position_subset.shape}")
print(f"Laps subset shape: {laps_subset.shape}")
print(f"Sessions subset shape: {sessions_subset.shape}")

def optimize_dataframe(df):
    
    # Create a copy of the dataframe
    df = df.copy()  

    # Convert int64 to int32
    for col in df.select_dtypes(include=["int64"]).columns:
        df.loc[:, col] = pd.to_numeric(df[col], downcast = "integer")  

    # Convert float64 to float32
    for col in df.select_dtypes(include=["float64"]).columns:
        df.loc[:, col] = pd.to_numeric(df[col], downcast = "float")

    # Convert object to category
    for col in df.select_dtypes(include=["object"]).columns:
        df.loc[:, col] = df[col].astype("category")

    return df

dataframes_subset = [optimize_dataframe(df) for df in dataframes_subset]

print(dataframes_subset[0].dtypes)

Driver subset shape: (200, 5)
Position subset shape: (200, 3)
Laps subset shape: (200, 3)
Sessions subset shape: (10, 5)
session_key       int64
driver_number     int64
full_name        object
team_name        object
headshot_url     object
dtype: object


In [30]:
# Merge DataFrames

# Start with the first DataFrame
merged_df = dataframes_subset[0]  
print(f"Initial dataframe shape: {merged_df.shape}")

for i in range(1, len(dataframes_subset)):  

    # Get rid of duplicate keys before each merge
    dataframes_subset[i] = dataframes_subset[i].drop_duplicates()

    # Determine common columns between the two DataFrames
    common_cols = list(set(merged_df.columns) & set(dataframes_subset[i].columns))
    
    # Ensure 'session_key' is always used as a key
    if "session_key" not in common_cols:
        common_cols.append("session_key")

    # Merge only on common columns
    merged_df = pd.merge(
        merged_df, dataframes_subset[i],
        on = common_cols,
        how = "inner"
    ).astype("category", errors = "ignore")

    merged_df = optimize_dataframe(merged_df)
    print(f"Merged Dataframe shape on {i}, {merged_df.shape}")

print(f"Final Merged DataFrame {merged_df.shape}:")
print(merged_df.columns)
merged_df["year"] = merged_df["date_start"].astype(str).str[:4]
merged_df = merged_df.drop(columns = ["date_start"])
print(merged_df.shape)
desired_order = ["session_key", "session_type", "position", "driver_number", "full_name", "team_name", "lap_number", "circuit_key", "location", "year", "headshot_url"]
final_df = merged_df[desired_order]
final_df

Initial dataframe shape: (200, 5)
Merged Dataframe shape on 1, (200, 6)
Merged Dataframe shape on 2, (200, 7)
Merged Dataframe shape on 3, (200, 11)
Final Merged DataFrame (200, 11):
Index(['session_key', 'driver_number', 'full_name', 'team_name',
       'headshot_url', 'position', 'lap_number', 'circuit_key', 'session_type',
       'location', 'date_start'],
      dtype='object')
(200, 11)


,session_key,session_type,position,driver_number,full_name,team_name,lap_number,circuit_key,location,year,headshot_url
0,7775,Qualifying,15,1,Max VERSTAPPEN,Red Bull Racing,6,149,Jeddah,2023,https://www.formula1.com/content/dam/fom-websi...
1,7775,Qualifying,20,2,Logan SARGEANT,Williams,8,149,Jeddah,2023,https://www.formula1.com/content/dam/fom-websi...
2,7775,Qualifying,19,4,Lando NORRIS,McLaren,4,149,Jeddah,2023,https://www.formula1.com/content/dam/fom-websi...
3,7775,Qualifying,10,10,Pierre GASLY,Alpine,23,149,Jeddah,2023,https://www.formula1.com/content/dam/fom-websi...
4,7775,Qualifying,1,11,Sergio PEREZ,Red Bull Racing,18,149,Jeddah,2023,https://www.formula1.com/content/dam/fom-websi...
...,...,...,...,...,...,...,...,...,...,...,...
195,10006,Race,7,44,Lewis HAMILTON,Ferrari,53,46,Suzuka,2025,https://media.formula1.com/d_driver_fallback_i...
196,10006,Race,14,55,Carlos SAINZ,Williams,53,46,Suzuka,2025,https://media.formula1.com/d_driver_fallback_i...
197,10006,Race,5,63,George RUSSELL,Mercedes,53,46,Suzuka,2025,https://media.formula1.com/d_driver_fallback_i...
198,10006,Race,3,81,Oscar PIASTRI,McLaren,53,46,Suzuka,2025,https://media.formula1.com/d_driver_fallback_i...


In [31]:
final_df = final_df[(final_df.full_name != "Sergio PEREZ") & (final_df.full_name != "Kevin MAGNUSSEN") & (final_df.full_name != "Daniel RICCIARDO") & (final_df.full_name != "ZHOU Guanyu") & (final_df.full_name != "Logan SARGEANT") & (final_df.full_name != "Nyck DE VRIES")]
final_df

,session_key,session_type,position,driver_number,full_name,team_name,lap_number,circuit_key,location,year,headshot_url
0,7775,Qualifying,15,1,Max VERSTAPPEN,Red Bull Racing,6,149,Jeddah,2023,https://www.formula1.com/content/dam/fom-websi...
2,7775,Qualifying,19,4,Lando NORRIS,McLaren,4,149,Jeddah,2023,https://www.formula1.com/content/dam/fom-websi...
3,7775,Qualifying,10,10,Pierre GASLY,Alpine,23,149,Jeddah,2023,https://www.formula1.com/content/dam/fom-websi...
5,7775,Qualifying,3,14,Fernando ALONSO,Aston Martin,18,149,Jeddah,2023,https://www.formula1.com/content/dam/fom-websi...
6,7775,Qualifying,2,16,Charles LECLERC,Ferrari,19,149,Jeddah,2023,https://www.formula1.com/content/dam/fom-websi...
...,...,...,...,...,...,...,...,...,...,...,...
195,10006,Race,7,44,Lewis HAMILTON,Ferrari,53,46,Suzuka,2025,https://media.formula1.com/d_driver_fallback_i...
196,10006,Race,14,55,Carlos SAINZ,Williams,53,46,Suzuka,2025,https://media.formula1.com/d_driver_fallback_i...
197,10006,Race,5,63,George RUSSELL,Mercedes,53,46,Suzuka,2025,https://media.formula1.com/d_driver_fallback_i...
198,10006,Race,3,81,Oscar PIASTRI,McLaren,53,46,Suzuka,2025,https://media.formula1.com/d_driver_fallback_i...


In [32]:
from pymongo import MongoClient

In [33]:
client = MongoClient("mongodb://localhost:27017/")
db = client["f1_db"]
collection = db["race_data"]

In [34]:
data_dict = final_df.to_dict("records")

In [35]:
collection.insert_many(data_dict)

InsertManyResult([ObjectId('67f8b18d30982c1832f2c7ad'), ObjectId('67f8b18d30982c1832f2c7ae'), ObjectId('67f8b18d30982c1832f2c7af'), ObjectId('67f8b18d30982c1832f2c7b0'), ObjectId('67f8b18d30982c1832f2c7b1'), ObjectId('67f8b18d30982c1832f2c7b2'), ObjectId('67f8b18d30982c1832f2c7b3'), ObjectId('67f8b18d30982c1832f2c7b4'), ObjectId('67f8b18d30982c1832f2c7b5'), ObjectId('67f8b18d30982c1832f2c7b6'), ObjectId('67f8b18d30982c1832f2c7b7'), ObjectId('67f8b18d30982c1832f2c7b8'), ObjectId('67f8b18d30982c1832f2c7b9'), ObjectId('67f8b18d30982c1832f2c7ba'), ObjectId('67f8b18d30982c1832f2c7bb'), ObjectId('67f8b18d30982c1832f2c7bc'), ObjectId('67f8b18d30982c1832f2c7bd'), ObjectId('67f8b18d30982c1832f2c7be'), ObjectId('67f8b18d30982c1832f2c7bf'), ObjectId('67f8b18d30982c1832f2c7c0'), ObjectId('67f8b18d30982c1832f2c7c1'), ObjectId('67f8b18d30982c1832f2c7c2'), ObjectId('67f8b18d30982c1832f2c7c3'), ObjectId('67f8b18d30982c1832f2c7c4'), ObjectId('67f8b18d30982c1832f2c7c5'), ObjectId('67f8b18d30982c1832f2c7

In [36]:
for doc in collection.find().limit(5):
    print(doc)

{'_id': ObjectId('67f8b18d30982c1832f2c7ad'), 'session_key': 7775, 'session_type': 'Qualifying', 'position': 15, 'driver_number': 1, 'full_name': 'Max VERSTAPPEN', 'team_name': 'Red Bull Racing', 'lap_number': 6, 'circuit_key': 149, 'location': 'Jeddah', 'year': '2023', 'headshot_url': 'https://www.formula1.com/content/dam/fom-website/drivers/M/MAXVER01_Max_Verstappen/maxver01.png.transform/1col/image.png'}
{'_id': ObjectId('67f8b18d30982c1832f2c7ae'), 'session_key': 7775, 'session_type': 'Qualifying', 'position': 19, 'driver_number': 4, 'full_name': 'Lando NORRIS', 'team_name': 'McLaren', 'lap_number': 4, 'circuit_key': 149, 'location': 'Jeddah', 'year': '2023', 'headshot_url': 'https://www.formula1.com/content/dam/fom-website/drivers/L/LANNOR01_Lando_Norris/lannor01.png.transform/1col/image.png'}
{'_id': ObjectId('67f8b18d30982c1832f2c7af'), 'session_key': 7775, 'session_type': 'Qualifying', 'position': 10, 'driver_number': 10, 'full_name': 'Pierre GASLY', 'team_name': 'Alpine', 'lap